# Fitting crescita COVID 19

In [71]:
#Importazione librerie operative
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import curve_fit
from ipywidgets import interact

In [72]:
#Lettura dati sperimentali via web
df = pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')

### Impostazione funzione di fitting - curva "logistica"
  
### $$ N(N_{tot},N_0,r,t) = \frac{N_{tot}N_0}{N_0+(N_{tot}-N_0)e^{-rt}}$$


In [73]:
def sigma(x,a,b,c):
    y=(a*b)/(b+(a-b)*np.exp(-c*x))
    return y

In [78]:
Nexp = df['totale_positivi']
a = df.count()
t = 1+np.arange(a['data'])
t1 = np.arange(70)
t2 = t1-1

In [79]:
df_sel=df.drop(['data','stato','note_it', 'note_en'],axis=1)
@interact(parametro=df_sel.columns)
def sel(parametro):
    fg=go.Figure()
    return fg.add_trace(go.Scatter(x=df.data ,y=df[parametro], mode='markers', marker_color = 'green'))

interactive(children=(Dropdown(description='parametro', options=('ricoverati_con_sintomi', 'terapia_intensiva'…

##### Fit dati sperimentali

In [80]:
popt, pcov = curve_fit(sigma, t, Nexp, maxfev=5000, method='trf', bounds=(0, [200000., 1500., 0.5]))
fit = sigma(t, *popt)
k, N0, r = popt
der = sigma(t1,*popt) - sigma(t2,*popt)
print('Ntotale=' ,int(popt[0])  ,'N0=' ,int(popt[1])  ,'r=' ,"%0.2f" % popt[2])

Ntotale= 107829 N0= 975 r= 0.15


### Plot risultati ottenuti

In [81]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=Nexp, mode='markers',name='n°casi totali', marker=dict(symbol=[200],color='red', size=8)))
fig.add_trace(go.Scatter(x=t1, y=sigma(t1,*popt), mode='lines',name='Previsione',marker_color='green'))
fig.update_layout(title='Crescita casi COVID 19', \
                  xaxis_title='giorni da 24 feb',yaxis_title='n° totale casi')

In [82]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=t, y=df['variazione_totale_positivi'], mode='markers',name='incremento casi/giorno', \
                         marker=dict(symbol=[200],color='red', size=10)))
fig.add_trace(go.Scatter(x=t1, y=der, mode='lines',name='Previsione',marker_color='blue'))
fig.update_layout(title='Incremento giornaliero positivi COVID 19', \
                  xaxis_title='giorni da 24 feb',yaxis_title='n° casi giornalieri')

In [ ]:
tab=df.drop(['stato','note_it', 'note_en'],axis=1)
tab